
# PyBaMM: A High-Level Overview

PyBaMM (Python Battery Mathematical Modelling) is an open-source battery simulation package written in Python. Our mission is to accelerate battery modelling research by providing open-source tools for multi-institutional, interdisciplinary collaboration. Broadly, PyBaMM consists of

- a framework for writing and solving systems of differential equations,
- a library of battery models and parameters, and
- specialized tools for simulating battery-specific experiments and visualizing the results.

Together, these enable flexible model definitions and fast battery simulations, allowing users to explore the effect of different battery designs and modeling assumptions under a variety of operating scenarios.

In [4]:
import pybamm

# Solving Battery models

PyBamm has three different base models:
- Single Particle Model (SPM)
- Single Particle Model with Electrolyte (SPMe)
- Doyle-Fuller-Newman (DFN) model

Simulating from any of these models is straightforward in Python. As a very first example, we will run the Doyle-Fuller-Newman (DFN) model with all the basic settings 

In [5]:
model = pybamm.lithium_ion.DFN()

The next step is to create a pybamm.Simulation to process the model and make it ready to be solved:

In [6]:
simulation = pybamm.Simulation(model)

The simulation can be solved by calling the `solve` method in the simulation object. The method takes an argument which should be the time interval to solve for in seconds (here we do 1 hour, as the default setup is a 1C discharge):

In [7]:
simulation.solve([0, 3600])

Now that the simulation has been solved, we can simply call the `plot` method to generate an interactive plot of the key variables:


In [8]:
simulation.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

# Comparing Models

We have seen how to run a simulation of the DFN model, but PyBaMM includes many other standard electrochemical models and makes comparing their outputs very easy. Let’s now compare the DFN model with the Single Particle Model (SPM) and the Single Particle Model with electrolyte (SPMe). We can leverage the functionality of Python lists to do so.

In [9]:
models = [
	pybamm.lithium_ion.SPM(),
	pybamm.lithium_ion.SPMe(),
	pybamm.lithium_ion.DFN(),
]
simulations = []
for model in models:
	simulation = pybamm.Simulation(model)
	simulation.solve([0, 3600])
	simulations.append(simulation)
pybamm.dynamic_plot(simulations)


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Changing the parameters

When running our simulations we have been using the default parameter values, which might not match the battery we want to simulate. PyBaMM has a number of in-built parameter sets (check the list [here](https://docs.pybamm.org/en/latest/source/api/parameters/parameter_sets.html)) that we can use. For example, to use the Chen et al (2020) parameter set we can use.

In [10]:
parameter_values = pybamm.ParameterValues("Chen2020")
print(parameter_values)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x000002B1D4064940>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x000002B1D40648B0>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initi

We can update the parameters of the model using this instance of `pybamm.ParameterValues`.

In [ ]:
parameter_values["Current function [A]"] = 2.0
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])

## Using input parameters

Pybamm's parameter sets contain all the parameters needed to run a simulation, but often you want to see how the solution changes with respect to a single parameter or a small subset of parameters. You can use input parameters to create a model that can be easily and efficiently solved again for different parameter values.

This is also very useful for parameter inference. Here we show an example using a parameter sweep.

In [14]:
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values["Current function [A]"] = "[input]"
model = pybamm.lithium_ion.SPM()
sim = pybamm.Simulation(model, parameter_values=parameter_values)

solutions = []
labels = []
for i in range(1, 5):
    sol = sim.solve([0, 3600], inputs={ "Current function [A]": i})
    solutions.append(sol)
    labels.append(f"Current = {i} [A]")
pybamm.dynamic_plot(solutions, labels=labels)
    



interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Submodels: Plug-and-play physics

One of the main features of PyBaMM is its modular structure that allows for plug and play models. At the core, all models in PyBaMM are built as a collection of submodels, where a submodel determines a specific subset of the physics. For example, the particle submodel would specify how lithium is transported in the particles. 

The full list of submodels can be found in the [PyBaMM docs](https://docs.pybamm.org/en/latest/source/api/models/submodels/index.html). You can check which submodels a given model uses by calling


In [15]:
model.submodels

{'external circuit': <pybamm.models.submodels.external_circuit.explicit_control_external_circuit.ExplicitCurrentControl at 0x2b1d529c040>,
 'porosity': <pybamm.models.submodels.porosity.constant_porosity.Constant at 0x2b1d529c0d0>,
 'Negative interface utilisation': <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full at 0x2b1d529c220>,
 'Positive interface utilisation': <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full at 0x2b1d529c310>,
 'negative particle mechanics': <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics at 0x2b1d529c340>,
 'positive particle mechanics': <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics at 0x2b1d529c3a0>,
 'negative primary active material': <pybamm.models.submodels.active_material.constant_active_material.Constant at 0x2b1d529c190>,
 'positive primary active material': <pybamm.models.submodels.active_material.constant_active_material.Constant at 0x2b1d529c3d0>,


## Thermal models

For example, we can consider the thermal models in PyBaMM. These models account for the changes in the temperature caused by the operation of the battery. The thermal models available in PyBaMM are:

* **Isothermal**: temperature stays constant.
* **Lumped**: the temperature is taken to be homogeneous in the battery, so only the average temperature is computed.
* **X-lumped**: the temperature is taken to be homogeneous across the thickness of the cell, but can vary in the directions parallel to the current collectors. Need to be used in conjunction with a current collector model.
* **X-full**: the temperature is allowed to vary across the thickness of the cell.

More information on the thermal models can be found in [the documentation](https://docs.pybamm.org/en/latest/source/examples/notebooks/models/thermal-models.html).

Below we compare the results of the DFN model with isothermal and x-full. What differences do you observe in the models?

In [16]:
thermal_options = ["isothermal", "x-full"]
solutions = []

for option in thermal_options:
	model = pybamm.lithium_ion.DFN(name=option, options={"thermal": option})
	simulation = pybamm.Simulation(model)
	solutions.append(simulation.solve([0, 3600]))

pybamm.dynamic_plot(
    solutions,
    output_variables=[
        "Negative particle surface concentration [mol.m-3]",
        "Electrolyte concentration [mol.m-3]",
        "Positive particle surface concentration [mol.m-3]",
        "Negative electrode potential [V]",
        "Electrolyte potential [V]",
        "Positive electrode potential [V]",
        "Current [A]",
        "Voltage [V]",
        "Cell temperature [K]",
    ],
  )

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

We observe that the temperature in the isothermal model remains constant, while for the x-full model the temperature rises almost 20 K. Despite being a spatial-dependent model, the temperature gradients are negligible, which is reasonable given how thin the cell is. We also observe that the voltage for the isothermal case is lower than for the temperature dependent one, given that at higher temperatures the overpotentials are smaller.

## Particle mechanics

As another example, we consider the models for particle mechanics. These models account for the deformation and cracking on the particles. The models available in PyBaMM are

* None: no mechanical effects included.
* Swelling only: accounts for the deformation of the particles in the lithiation-delithiation cycle.
* Swelling and cracking: accounts for the swelling and also the crack formation on the particle surface.

Here we run the DFN model with swelling in the negative electrode and swelling and cracking in the positive electrode. Plot the relevant variables, and recall that you can find the list of variables to plot by typing `model.variable_names()`. What do you observe in the model?


In [17]:
model = pybamm.lithium_ion.DFN(
    options={"particle mechanics": ("swelling only", "swelling and cracking")}
)
parameter_values = pybamm.ParameterValues("Ai2020")
simulation = pybamm.Simulation(model, parameter_values=parameter_values)
solution = simulation.solve([0, 3600])

solution.plot([
    "Negative particle surface radial stress [Pa]",
    "Negative particle surface tangential stress [Pa]",
    "Negative particle surface displacement [m]",
	"Negative particle crack length [m]",
    "Positive particle surface radial stress [Pa]",
    "Positive particle surface tangential stress [Pa]",
    "Positive particle surface displacement [m]",
    "Positive particle crack length [m]",
])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

A few key observations are that the surface radial stress is always zero. As expected, there is no cracking in the negative electrode (we did not enable that option) but there is cracking in the positive one. 